In [86]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer, MinMaxScaler, LabelBinarizer, LabelEncoder

In [121]:
data = pd.read_excel('dataset/titanic3.xls')
data.drop(labels=['name', 'body', 'home.dest', 'ticket', 'boat'],axis=1, inplace=True)
data = data.dropna()
data_x, data_y = data.drop(labels='survived', axis=1), data['survived']
data_y = np.array(y).reshape(-1,1)

In [123]:
data_x_numerical = data_x.drop(labels=['sex', 'cabin', 'embarked'], axis=1)
scaler = MinMaxScaler(feature_range=(0,1))
data_x_numerical[['age', 'fare']] = scaler.fit_transform(data_x_numerical[['age', 'fare']])
data_x_numerical

,pclass,age,sibsp,parch,fare
0,1,0.355110,0,0,0.412503
1,1,0.000000,1,2,0.295806
2,1,0.013698,1,2,0.295806
3,1,0.367755,1,2,0.295806
4,1,0.304531,1,2,0.295806
5,1,0.595363,0,0,0.051822
6,1,0.785037,1,0,0.152164
7,1,0.481559,0,0,0.000000
8,1,0.658588,2,0,0.100481
10,1,0.582718,1,0,0.444099


In [136]:
data_x_categorical = data_x[['sex', 'cabin', 'embarked']]
binarizer = LabelBinarizer()
encoder = LabelEncoder()
data_x_categorical['sex'] = binarizer.fit_transform(data_x_categorical['sex'])
data_x_categorical['cabin'] = encoder.fit_transform(data_x_categorical['cabin'])
data_x_categorical['embarked'] = encoder.fit_transform(data_x_categorical['embarked'])
#data_x_categorical['cabin'] = scaler.fit_transform(data_x_categorical['cabin'])
data_x_categorical

C:\Users\Diogo\AppData\Local\conda\conda\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Diogo\AppData\Local\conda\conda\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Diogo\AppData\Local\conda\conda\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,sex,cabin,embarked
0,0,38,2
1,1,73,2
2,0,73,2
3,1,73,2
4,0,73,2
5,1,139,2
6,0,135,2
7,1,12,2
8,0,56,2
10,1,90,0


# Entendendo o dataset

Para iniciar, deve-se analisar os atributos de entrada do dataset, seus tipos e o atributo alvo (label/rótulo). Isso pode ser feito através do Pandas, biblioteca de Python específica para análise e preprocessamento de dados. Isso pode ser feito através da biblioteca Pandas, que permite analisar e preprocessar dados. 

In [5]:
# Passo 1 - Leitura do dataset 
train = None
test  = None
# Passo 2 - Separar atributos e classes
X_train = None
y_train = None
X_test =  None
y_test =  None


# Identificando problemas

Mesmo com uma base de dados previamente definida, muitas vezes existem problemas que não foram tratados nela, sendo necessário analisar manualmente. Dentre os problemas, para bases de texto, três são bastante comuns:
* Instâncias com informações faltando (NaN) para determinados atributos
* Dados discrepantes e outliers
* Dados desbalanceados

Dessa forma, deve-se itentificar a presença desses problemas no dataset. 
- Dica 1: utilize funções das bibliotecas do Pandas para os dois primeiros problemas.  
- Dica 2: visualize a distribuição de instâncias por classe através de bibliotecas gráficas de Python (e.g. matplob, seaborn e pyplot).

# Solucionando Problemas

Os problemas podem ser solucionados utilizando algumas medidas estatísticas como média, mediana e moda para substituir as informações que estão faltando e dados discrepantes, ou excluir as instâncias. Além disso, quanto mais balanceado o dataset, menos propensa estará a rede a cometer erros de generalização. A resolução pode ser feita das duas formas utilizando o Pandas. 

# Correlação dos atributos

Uma vez que o dataset foi analisado e possíveis erros foram corrigidos, é importante também verificar a correlação dos atributos. A correlação é uma métrica estatística que mede a associação entre os atributos. Caso existam atributos altamente correlacionados, pode-se excluir alguns deles, permanecendo apenas um dos atributos correlacionados.

Dessa forma, nessa etapa, calcule a correlação entre os atributos. Considere utilizar algumas formas de visualização para melhor interpretação dos resultados.

# Implementando a Rede Neural

Uma vez que a base de dados foi analisada e possíveis problemas foram corrigidos, pode-se implementar a rede neural. Para esse problema, é necessário uma arquitetura multicamada MLP de no máximo 6 camadas escondidas, recomendando-se a implementação das camadas de forma gradual. Da mesma forma, cada camada deverá conter no máximo 30 neurônios. Além disso, recomenda-se a utilização de técnicas de regularização como Dropout, normalização do batch e normalização L2.

Utilize o otimizador de sua escolha para o treinamento por batch, aumente a quantidade de epochs e batch também gradativamente, não é necessário uma quantidade muito alta para resolução do problema. Ao final, mostre a curva de treinamento e a matriz de confusão obtida para o problema. 